In [1]:
import pandas as pd
import numpy as np

In [2]:
# Список с кодом товара, наименованием SKU и его долей в плане относительно каждого квартала
dfu_quart= pd.read_excel('Распределение косметология по DFU.xlsx', sheet_name = 'dfu')
# План в рублях на каждый ФО в разбивке по кварталам
budget = pd.read_excel('Распределение косметология по DFU.xlsx', sheet_name = 'budget_rub')
# Список территориальных менеджеров с их долями на ФО и квартал
share_fo = pd.read_excel('Распределение косметология по DFU.xlsx',sheet_name = 'share_fo')
# Список территориальных менеджеров с привязкой к региону и долей
share_reg = pd.read_excel('Распределение косметология по DFU.xlsx',sheet_name = 'share_reg')
# Цена каждого SKU
price = pd.read_excel('Распределение косметология по DFU.xlsx',sheet_name = 'price')

col = ['Квартал', 'Дата', 'Доля']
share_month = [['1Q', '01.01.2025', 0.17],
              ['1Q', '01.02.2025', 0.3],
              ['1Q', '01.03.2025', 0.53],
              ['2Q', '01.04.2025', 0.4],
              ['2Q', '01.05.2025', 0.4],
              ['2Q', '01.06.2025', 0.2],
              ['3Q', '01.07.2025', 0.15],
              ['3Q', '01.08.2025', 0.35],
              ['3Q', '01.09.2025', 0.5],
              ['4Q', '01.10.2025', 0.2],
              ['4Q', '01.11.2025', 0.4],
              ['4Q', '01.12.2025', 0.4]]
# Таблица с долями каждого месяца относительно квартала
fasing = pd.DataFrame(data = share_month, columns = col)

In [3]:
display(dfu_quart.head(3))
display(budget.head(3))
display(share_fo.head(3))
display(share_reg.head(3))
display(price.head(3))
display(fasing.head(3))

,Продукт: DFU код,Продукт: DFU описание,1Q,2Q,3Q,4Q
0,00-00017334,SKU_1,0.061856,0.064408,0.070219,0.066632
1,00-00017331,SKU_2,0.061856,0.064408,0.070219,0.066632
2,00-00017333,SKU_3,0.033550,0.031799,0.032974,0.036744


,ФО,1Q,2Q,3Q,4Q
0,ЦФО,6.554252e+07,6.614033e+07,9.103405e+07,2.425517e+07
1,СЗФО,2.442958e+07,1.882202e+07,1.565058e+07,8.250284e+07
2,УФО,9.124607e+07,6.357971e+06,6.941112e+07,3.156565e+07


,ТМ,ФО,1Q,2Q,3Q,4Q
0,ТМ_1,СФО,1.000,1.000,1.000,1.000
1,Вакансия_1,ДВФО,1.000,1.000,1.000,1.000
2,ТМ_2,ЮФО,0.334,0.334,0.334,0.334


,ТМ,Регион,ФО,Доля
0,ТМ_1,Алтайский край,СФО,0.09
1,Вакансия_1,Амурская обл,ДВФО,0.10
2,ТМ_17,Архангельская обл,СЗФО,0.05


,Продукт: DFU код,Продукт: DFU описание,Цена
0,00-00042670,SKU_11,2569
1,00-00042671,SKU_9,2509
2,00-00052371,SKU_12,4211


,Квартал,Дата,Доля
0,1Q,01.01.2025,0.17
1,1Q,01.02.2025,0.30
2,1Q,01.03.2025,0.53


In [4]:
budget_melt = pd.melt(budget, id_vars = 'ФО',  value_name=  'Бюджет', var_name = 'Квартал')

In [5]:
dfu_melt = pd.melt(dfu_quart, id_vars = ['Продукт: DFU код','Продукт: DFU описание'], 
                   value_name=  'Доля', var_name = 'Квартал')

In [6]:
df_new = dfu_melt.merge(budget_melt, on = 'Квартал')
df_new['Бюджет ФО'] = df_new['Доля']*df_new['Бюджет']
df_new = df_new[['Продукт: DFU код', 'Продукт: DFU описание', 'Квартал', 'ФО', 'Бюджет ФО']]

In [7]:
tm_melt = pd.melt(share_fo, id_vars = ['ТМ','ФО'], value_name=  'Доля', var_name = 'Квартал')

In [8]:
df_new = df_new.merge(tm_melt, on = ['ФО', 'Квартал'])
df_new['Бюджет ТМ'] = df_new['Бюджет ФО'] * df_new['Доля']
del df_new['Доля']

In [9]:
df_new = df_new.merge(share_reg, on = ['ТМ', 'ФО'])
df_new['Бюджет Регион'] = df_new['Бюджет ТМ'] * df_new['Доля']
del df_new['Доля']

In [10]:
df_new = df_new.merge(fasing, on = 'Квартал', how='left')

In [11]:
df_new['План месяц, руб'] = df_new['Бюджет Регион'] * df_new['Доля']
del df_new['Доля']

In [12]:
plan_rub = df_new.merge(price, on = ['Продукт: DFU код', 'Продукт: DFU описание'], how='left')

In [13]:
plan_rub['План месяц, уп'] = plan_rub['План месяц, руб'] / plan_rub['Цена']

In [14]:
print(plan_rub['План месяц, уп'].sum())
print(plan_rub['План месяц, руб'].sum())
print(budget_melt['Бюджет'].sum()) # Сверяем изначальную сумму бюджета в 
#рублях с получившимся после вычислений планом в рублях. Рассчитано верно

361436.4365355388
1386099193.6786385
1386099193.6786382


In [15]:
# Функция для округления числа до целого. Стандартный round() срабатывает не так, как требуется
def user_round(x):
    if x - int(x) > 0.5:
        return int(x) + 1
    else:
        return int(x)

In [16]:
# План в упаковках должен быть целым числом, поэтому применяем функцию к столбцу с планами уп
plan_rub['План месяц, уп'] = plan_rub['План месяц, уп'].map(user_round)

In [17]:
plan_rub['План месяц, уп'].sum() # Проверка

361274

In [18]:
display(plan_rub.head(5)) # Смотрим на итоговую таблицу

,Продукт: DFU код,Продукт: DFU описание,Квартал,ФО,Бюджет ФО,ТМ,Бюджет ТМ,Регион,Бюджет Регион,Дата,"План месяц, руб",Цена,"План месяц, уп"
0,00-00017334,SKU_1,1Q,ЦФО,4.054225e+06,ТМ_3,810845.057351,Владимирская обл,16216.901147,01.01.2025,2756.873195,3700,1
1,00-00017334,SKU_1,1Q,ЦФО,4.054225e+06,ТМ_3,810845.057351,Владимирская обл,16216.901147,01.02.2025,4865.070344,3700,1
2,00-00017334,SKU_1,1Q,ЦФО,4.054225e+06,ТМ_3,810845.057351,Владимирская обл,16216.901147,01.03.2025,8594.957608,3700,2
3,00-00017334,SKU_1,1Q,ЦФО,4.054225e+06,ТМ_3,810845.057351,Ивановская обл,40542.252868,01.01.2025,6892.182987,3700,2
4,00-00017334,SKU_1,1Q,ЦФО,4.054225e+06,ТМ_3,810845.057351,Ивановская обл,40542.252868,01.02.2025,12162.675860,3700,3


In [19]:
plan_rub.to_excel('План в рублях косметология.xlsx') #Сохраняем итоговый план